In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV



categories = ['comp.sys.mac.hardware', 'rec.sport.hockey']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories, remove=('headers', 'footers', 'quotes'))
newsgroups_test  = fetch_20newsgroups(subset='test',  categories=categories, remove=('headers', 'footers', 'quotes'))


from sklearn.pipeline import Pipeline
classifier = Pipeline([ ('vect', CountVectorizer(stop_words='english')),
                        ('tfidf', TfidfTransformer()),
                        ('knn', KNeighborsClassifier())
                        ])


parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'knn__n_neighbors': list(range(100,300))
             }

gs_classifier = GridSearchCV(classifier, parameters, n_jobs=-1)
gs_classifier = gs_classifier.fit(newsgroups_train.data, newsgroups_train.target)

print(gs_classifier.best_score_)
print(gs_classifier.best_params_)

In [3]:
import pandas as pd

data = pd.read_csv(r'trip_advisor_df.csv')
data.head()

,Unnamed: 0,title,url,rating,no_ratings,desc,price
0,0,Best of Ubud Tour with Jungle Swing,/AttractionProductReview-g608496-d12955335-Bes...,5.0,865,"Hit the Ubud highlights in just a day, without...","2,611.78"
1,1,Bali Full-Day Traditional Village Sightseeing ...,/AttractionProductReview-g1219108-d11456045-Ba...,5.0,"1,154",This sightseeing tour is for travelers looking...,"2,089.43"
2,2,Bali Day Tour - Exploring The Most Scenic Spots,/AttractionProductReview-g469404-d15696676-Bal...,5.0,327,Home to the colorfull a part of Bali most trav...,"1,619.31"
3,3,Ubud - Jungle Swing - Monkey Forest - Waterfal...,/AttractionProductReview-g469404-d16741326-Ubu...,5.0,73,Panning your tour of highlights can be difficu...,565.71
4,4,Bali Quad Bike Adventure,/AttractionProductReview-g608496-d11464623-Bal...,5.0,270,"The wild jungle trails, bamboo forests, and ri...","2,193.90"


In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

vectorizer = TfidfVectorizer()
count_vect = CountVectorizer()
X = count_vect.fit_transform(data['desc'])
Y = vectorizer.fit_transform(data['desc'])

a = count_vect.get_feature_names()
b = X.toarray().sum(axis=0)
c = vectorizer.idf_

d = pd.DataFrame(a, columns=['word'])
d['count'] = b
d['idf'] = c
d.sort_values('count')

,word,count,idf
1458,lines,1,7.213608
1565,mealea,1,7.213608
1563,mdpl,1,7.213608
1562,maya,1,7.213608
1559,maximal,1,7.213608
1557,matters,1,7.213608
1556,mastering,1,7.213608
1553,mas,1,7.213608
1548,marinsport,1,7.213608
1544,manual,1,7.213608


In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

documents = ["This little kitty came to play when I was eating at a restaurant.",
             "Merley has the best squooshy kitten belly.",
             "Google Translate app is incredible.",
             "If you open 100 tab in google you get a smiley face.",
             "Best cat photo I've ever taken.",
             "Climbing ninja cat.",
             "Impressed with google map feedback.",
             "Key promoter extension for Google Chrome."]

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(data['desc'])

true_k = 3
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

print("\n")
print("Prediction")

Y = vectorizer.transform(["chrome browser to open."])
prediction = model.predict(Y)
print(prediction)

Y = vectorizer.transform(["My cat is hungry."])
prediction = model.predict(Y)
print(prediction)

print(metrics.classification_report(Y, prediction))

Top terms per cluster:
Cluster 0:
 balinese
 cooking
 cuisine
 class
 learn
 indonesian
 local
 culture
 ballet
 authentic
Cluster 1:
 nusa
 day
 penida
 tour
 bali
 beach
 snorkeling
 ll
 trip
 island
Cluster 2:
 bali
 tour
 private
 ubud
 driver
 experience
 guide
 temple
 adventure
 day


Prediction
[2]
[2]


ValueError: Classification metrics can't handle a mix of multilabel-indicator and binary targets